In [2]:
import win32com.client
import subprocess
import time

In [ ]:
# open SAP Logon
subprocess.Popen(r"C:\\Program Files (x86)\SAP\\FrontEnd\SAPgui\saplogon.exe")
time.sleep(5)
SapGuiAuto = win32com.client.GetObject('SAPGUI')
application = SapGuiAuto.GetScriptingEngine
connection = application.OpenConnection("USDEV",True)

# login
session = connection.Children(0)
session.findbyId("wnd[0]").maximize
session.findById("wnd[0]/usr/txtRSYST-MANDT").text = "800" # Client
session.findById("wnd[0]/usr/txtRSYST-BNAME").text = "JESAI"    # User
session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = "jaisai2025"  # Password
session.findById("wnd[0]/usr/txtRSYST-LANGU").text = "EN"   # Language
session.findById("wnd[0]").sendVKey(0) # Enter
time.sleep(1)

In [3]:
# navigate to transaction code
session.findById("wnd[0]/tbar[0]/okcd").text = "/nMB52" # Transaction code
session.findById("wnd[0]").sendVKey(0) # Enter
time.sleep(1)

In [4]:
# Entering Plant Number
session.findById("wnd[0]/usr/ctxtWERKS-LOW").text = "9100"
time.sleep(1)
session.findById("wnd[0]").sendVKey(8) # F8
time.sleep(1)


In [5]:
# Expporting to Excel
# session.findById("wnd[0]/tbar[1]/btn[43]").press # why its not pressing
# session.findById("wnd[0]").sendVKey(1047)
alv_ids = [
    "wnd[0]/usr/cntlGRID1/shellcont/shell",
    "wnd[0]/usr/cntlALV_GRID/shellcont/shell",
    "wnd[0]/usr/cntlRESULT_LIST/shellcont/shell",
]
grid = None
for pid in alv_ids:
    try:
        grid = session.findById(pid)
        break
    except:
        pass
if grid is None:
    raise Exception("ALV grid not found. Start recorder on the export click to grab the exact path.")

# give focus (important for some GUIs) and press Export-to-Excel
grid.setFocus()
# grid.pressToolbarButton("&XXL")          # Export → Spreadsheet
grid.selectContextMenuItem("&XXL")     

com_error: (-2147352567, 'Exception occurred.', (613, 'SAP Frontend Server', 'The method got an invalid argument.', None, 0, 0), None)